# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 22
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr22.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr22.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 170 170

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000177663,ENSG00000131100,ENSG00000099968,ENSG00000015475,ENSG00000243156,ENSG00000215193,ENSG00000183785,ENSG00000184979,ENSG00000100033,ENSG00000100075
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177663,1.000000000,-0.07717207,0.07890063,-0.03464399,-0.008794338,0.03595470,0.001726432,0.095401984,0.03910270,0.32156489
ENSG00000131100,-0.077172067,1.00000000,-0.25827894,0.15425866,-0.102483855,-0.03833009,-0.069450615,-0.077466787,-0.25148724,-0.36351196
ENSG00000099968,0.078900627,-0.25827894,1.00000000,-0.15236055,0.151567921,0.06397927,-0.028284257,0.094149609,0.02634172,0.23893616
ENSG00000015475,-0.034643993,0.15425866,-0.15236055,1.00000000,-0.075705611,-0.02813188,-0.160490888,0.014910651,-0.14498552,-0.06459698
ENSG00000243156,-0.008794338,-0.10248385,0.15156792,-0.07570561,1.000000000,-0.05797458,-0.077885080,0.010540094,0.03839897,0.01825553
ENSG00000215193,0.035954696,-0.03833009,0.06397927,-0.02813188,-0.057974580,1.00000000,0.007340717,-0.002354151,0.07588282,0.17708923


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 17

[1] -0.02527242 -0.06806841 -0.08941870 -0.11372257 -0.12507488 -0.18823897
 [7] -0.21172393 -0.22891750 -0.25600744 -0.29144790 -0.34729449 -0.39115611
[13] -0.52388628 -0.69599070 -0.95242435 -1.41858514 -2.44439322

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 170 170

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000177663,ENSG00000131100,ENSG00000099968,ENSG00000015475,ENSG00000243156,ENSG00000215193,ENSG00000183785,ENSG00000184979,ENSG00000100033,ENSG00000100075
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177663,1.000000000,-0.07717207,0.07890063,-0.03464399,-0.008794338,0.03595470,0.001726432,0.095401984,0.03910270,0.32156489
ENSG00000131100,-0.077172067,1.00000000,-0.25827894,0.15425866,-0.102483855,-0.03833009,-0.069450615,-0.077466787,-0.25148724,-0.36351196
ENSG00000099968,0.078900627,-0.25827894,1.00000000,-0.15236055,0.151567921,0.06397927,-0.028284257,0.094149609,0.02634172,0.23893616
ENSG00000015475,-0.034643993,0.15425866,-0.15236055,1.00000000,-0.075705611,-0.02813188,-0.160490888,0.014910651,-0.14498552,-0.06459698
ENSG00000243156,-0.008794338,-0.10248385,0.15156792,-0.07570561,1.000000000,-0.05797458,-0.077885080,0.010540094,0.03839897,0.01825553
ENSG00000215193,0.035954696,-0.03833009,0.06397927,-0.02813188,-0.057974580,1.00000000,0.007340717,-0.002354151,0.07588282,0.17708923


,ENSG00000177663,ENSG00000131100,ENSG00000099968,ENSG00000015475,ENSG00000243156,ENSG00000215193,ENSG00000183785,ENSG00000184979,ENSG00000100033,ENSG00000100075
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177663,1.000000000,-0.05991754,0.05975452,-0.03508067,0.007113431,0.05887898,0.036744097,0.047181653,-0.007069027,0.11536215
ENSG00000131100,-0.059917544,1.00000000,-0.15897191,0.12348569,-0.053720689,-0.02743585,-0.041079838,-0.044738824,-0.095175472,-0.15029619
ENSG00000099968,0.059754519,-0.15897191,1.00000000,-0.10466340,0.058608652,0.03259883,0.009275465,0.065200066,0.045277056,0.13012888
ENSG00000015475,-0.035080675,0.12348569,-0.10466340,1.00000000,-0.036701475,-0.01775967,-0.063091232,0.002945157,-0.075655511,-0.06769240
ENSG00000243156,0.007113431,-0.05372069,0.05860865,-0.03670147,1.000000000,-0.02518564,-0.045101643,0.002223870,0.022624437,0.02145516
ENSG00000215193,0.058878984,-0.02743585,0.03259883,-0.01775967,-0.025185638,1.00000000,0.018035075,0.014437230,0.012250985,0.07436558


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)